In [3]:
import os
import numpy as np
import pandas as pd
from itertools import combinations



def make_table(path):
    # read file names(id)
    list_of_files = os.listdir(path) # len=560
    index_list = []
    for file_name in list_of_files:
        index_list.append(int(file_name.removesuffix('.txt')))

    # read file content
    titles = []
    texts = []
    notices = []
    for file in list_of_files:
        f = open("{}/{}".format(path, file), "r")
        temp = f.readlines()
        titles.append(temp[0])
        texts.append(temp[1])
        try:
            notices.append(temp[2])
        except:
            notices.append('')
        f.close()

    # make general table
    df_dirty = pd.DataFrame(
        {'ID': index_list,
         'title': titles,
         'text': texts,
         'notice': notices
        })
    return df_dirty



def make_data(dataType, table):
    
    # make text_pairs, data scope can be changed, I only include "title" here
    # number of text_pairs = Cn取2, train:n=560; test:n=421
    combs = combinations(table[['ID', 'title']].values, r=2)
    combs = pd.DataFrame([[s for s in comb] for comb in combs], columns=['left', 'right'])
    combs[['id_left', 'left']] = combs['left'].to_list()
    combs[['id_right', 'right']] = combs['right'].to_list()

    if dataType=='train':
        # make label set
        labels = pd.read_csv('TrainLabel.csv')
        pairs = set()
        for _, row in labels.iterrows():
            temp = frozenset([row['Test'], row['Reference']])
            if temp not in pairs:
                pairs.add(temp)

        # mark label on text_pairs
        combs['label'] = 'unlike'
        for ind, row in combs.iterrows():
            if {row['id_left'], row['id_right']} in pairs:
                combs.at[ind, 'label'] = 'like'
    
    combs.to_csv("{}.tsv".format(dataType), sep="\t", index=False)            
    return combs


In [ ]:
table_train = make_table("./dataTrainComplete")
data_train = make_data('train', table_train)

table_test = make_table("./dataPublicComplete")
data_test = make_data('test', table_test)

In [4]:
table_train.head(10)

,ID,title,text,notice
0,289,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,行政院農業委員會臺東區農業改良場（以下簡稱臺東場）表示，目前轄區水稻生長情形多已達孕穗期至抽...,本病害防治時機，應把握抽穗前及齊穗期進行穗稻熱病預防性施藥，建議於抽穗前3~5天完成施藥，若...
1,504,可危害多種作物的木瓜秀粉介殼蟲發生嚴重，籲請農民加強防治\n,木瓜秀粉介殼蟲為木瓜新害蟲，除對木瓜造成嚴重危害外，對其它農作物蔬菜、果樹、花卉及雜草景觀植...,防治時機是初發現該蟲時應將受害枝條剪除並密封於塑膠袋中移出果園，藥劑防治可參考木瓜其它介殼蟲...
2,262,關心肺炎疫情，也要留心西瓜炭疽病已零星發生，籲請農友及早防治。臺中區農改場、防檢局及田邊好幫...,最近天氣逐漸轉熱且氣候變化劇烈，中部地區一期作西瓜目前進入開花結果期，行政院農業委員會臺中區...,高濕度是西瓜炭疽病最容易傳播的環境，尤其本病害傳播迅速，所以建議農友在田間病徵出現時應立即進...
3,276,臺東地區水稻瘤野螟（縱捲葉蟲）恐爆發危害，臺東場籲請農友加強防治。\n,臺東縣第二期水稻正值分蘗盛期至孕穗期，臺東市、鹿野鄉等部分地區發現水稻瘤野螟初期危害，為避免...,一、清除田邊雜草，減少成蟲棲息場所，以降低遷入蟲源。二、請合理使用氮肥切忌偏重氨肥，因瘤野螟...
4,510,苗栗水稻好發白葉枯病，籲請農友加強防治作業。苗栗縣政府、防檢局及田邊好幫手關心您。\n,苗栗縣水稻正值分蘗盛期，為水稻白葉枯病之好發期間，目前可見田區發生葉緣波浪狀之枯黃病徵且今年...,防治對策：1.常發病地區及風大地區避免種植感病品種。2.勿用手拔及剪除水稻葉尖，以避免病原菌...
5,921,發佈水稻葉稻熱病疫情警報\n,水稻病蟲害發生警報中華民國九十六年四月二十日第二號行政院農業委員會桃園區農業改良場桃園縣新屋...,\n
6,1292,冬末、初春種植香蕉組培苗易發生香蕉嵌紋病及萎縮病危害，香蕉所籲請農友及時強化防治工作。\n,一、香蕉組織苗定植田間初期，因植株較小，易受棉蚜及蕉蚜兩媒介帶毒蚜蟲所傳播之香蕉嵌紋病及萎縮...,三、預防方法：(一)、疫區宜儘速拔除病株，減少感染源，並補植健康成熟度大之吸芽。規劃新植組織...
7,29,發布水稻葉稻熱病防治警報，苗栗場籲請農友參考警報內容注意防治。\n,一、病蟲害別：水稻葉稻熱病二、防治地點、時期：縣市別鄉鎮別防治時期苗栗縣各鄉鎮市稻作栽培區發...,一、防治藥劑及方法：1.40%甲基多保淨水懸劑每公頃每次用藥量1公斤2.20%嘉賜三賽唑可濕...
8,1331,水稻飛蝨類害蟲密度攀升，白葉枯病零星出現，籲請農友加強防治。雲林縣政府、防檢局及田邊好幫手關...,雲嘉南地區水稻正值孕穗期，水稻乳熟期至糊熟期是飛蝨類害蟲危害最明顯的時間，臺南地區已發現受飛...,化學防治請選用核准登記及推薦使用於水稻之藥劑，藥劑種類可參考防檢局農藥資訊服務網(https...
9,1319,請蕉農務必採行正確香蕉葉部病害防治工作，俾確保蕉株良好發育、蕉果品質及產量。香蕉研究所、防檢...,近期時有細雨，加上早晚溫度低、露水量多，有利香蕉葉部病菌之感染及蔓延，造成葉片嚴重發病及枯萎...,一、噴藥前宜先行清園，並割除下方嚴重受害之病葉(發病超過30%之枯葉)，以減少感染源。進入終...


In [7]:
data_train.loc[data_train['label']=='like'].head(10)

,left,right,id_left,id_right,label
68,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,籲請農友注意防治穗稻熱病。臺南市政府、防檢局及田邊好幫手關心您。\n,289,1041,like
90,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,花蓮、宜蘭地區作物病蟲害疫情警報第2號\n,289,891,like
148,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,花蓮區水稻陸續進入分蘗終期或進入抽穗期，花蓮縣動植物防疫所提醒農友把握防治時機，避免穗稻熱病...,289,188,like
176,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,縣內已有零星稻熱病發生，籲請農友加強稻熱病防範工作。宜蘭縣政府、防檢局及田邊好幫手關心您。\n,289,809,like
236,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,嘉義市已發現水稻稻熱病發生，籲請農友掌握防治時機。嘉義市政府、防檢局及田邊好幫手關心您\n,289,1023,like
267,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,稻熱病進入好發季節，防檢局籲請農友加強防治\n,289,1005,like
276,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,雨後適合稻熱病發生，請持續進行監測並指導農民防治\n,289,988,like
281,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,本所近日監測縣內水稻田區，發現已有零星穗稻熱病發生，籲請提醒轄內農民加強穗稻熱病防範工作，宜...,289,817,like
305,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,提早入梅，請加強防範水稻稻熱病發生\n,289,1010,like
390,連日降雨且溫差大濕度高，請農友加強防範穗稻熱病，避免產量損失。\n,新聞稿-稻熱病進入好發季節，防檢局籲請農友加強防範\n,289,1000,like


In [26]:
key_chem = pd.read_excel("./Keywords/02chem.list.xlsx", header=None, index_col=False)
key_crop = pd.read_excel("./Keywords/02crop.list.xlsx", header=None, index_col=False)
key_pest = pd.read_excel("./Keywords/02pest.list.xlsx", header=None, index_col=False)

In [33]:
def make_key_lists(df):
    output = []
    num_cols = df.shape[1]
    for ind, row in df.iterrows():
        temp = [x for x in list(row) if pd.isnull(x) == False]
        temp.sort(key=len, reverse=True)
        output.append(temp)
    output = sorted(output, key=lambda x: len(x[0]), reverse=True)
    return output

list_chem = make_key_lists(key_chem)
list_crop = make_key_lists(key_crop)
list_pest = make_key_lists(key_pest)

In [34]:
list_pest

[['太平洋臀紋粉介殼蟲'],
 ['番茄黃化捲葉病毒', '番茄捲葉病'],
 ['番茄斑點萎凋病毒'],
 ['瓜類退綠黃化病毒'],
 ['霍香薊黃脈病毒'],
 ['褪綠斑駁病毒病'],
 ['臺灣本土黑螞蟻', '琉璃蟻'],
 ['細菌性黑斑病', '黑斑病', '鬼頭病', '輪斑病'],
 ['水稻縞葉枯病'],
 ['水稻水象鼻蟲'],
 ['番茄嵌紋病毒'],
 ['胡瓜嵌紋病毒'],
 ['馬鈴薯病毒Y'],
 ['細菌性條斑病'],
 ['二點小綠葉蟬'],
 ['細菌性斑點病'],
 ['細菌性角斑病', '臭葉'],
 ['柑橘窄胸天牛'],
 ['南瓜捲葉病毒'],
 ['白輪盾介殼蟲'],
 ['地中海果實蠅'],
 ['路易氏始葉螨'],
 ['斜紋夜盜蟲', '斜紋夜蛾', '行軍蟲', '黑肚蟲', '夜盜蟲', '黑蟲'],
 ['稻縱捲葉蟲', '縱捲葉蟲', '瘤野螟', '捲葉蟲', '葉尾蟲'],
 ['東方果實蠅', '蜂仔'],
 ['草狀矮化病'],
 ['皺縮矮化病'],
 ['壯鋏普癭蚋'],
 ['咖啡木蠹蛾'],
 ['台灣黃毒蛾'],
 ['小白紋毒蛾'],
 ['草地貪夜蛾', '秋行軍蟲'],
 ['胡麻葉枯病'],
 ['花姬捲葉蛾'],
 ['神澤氏葉螨'],
 ['葉鞘腐敗病'],
 ['柑橘赤葉螨'],
 ['黑條萎縮病'],
 ['秀粉介殼蟲'],
 ['黑點根腐病'],
 ['番茄斑潛蠅'],
 ['番茄病毒病'],
 ['線蟲白尖病'],
 ['亞洲潛葉蠅'],
 ['番茄潛旋蛾'],
 ['臺灣花薊馬'],
 ['立枯絲核菌'],
 ['假莖象鼻蟲'],
 ['球莖象鼻蟲'],
 ['疣胸琉璃蟻'],
 ['甜菜夜蛾', '蔥仔管蟲', '管仔蟲'],
 ['白背飛蝨'],
 ['荔枝椿象', '臭青龜仔', '臭屁蟲', '臭椿象', '荔椿'],
 ['白葉枯病'],
 ['穗稻熱病', '吊穗', '吊狗'],
 ['葉稻熱病'],
 ['小黃薊馬', '茶黃薊馬'],
 ['黑腳煙仔', '煤煙病', '煤污病'],
 ['灰象鼻蟲'],
 ['爛葉仔病', '晚疫病', '黑骨病', '黑腳病', '黑骨'],
 ['銀葉粉蝨', '白蒼蠅'],
 ['甘藷蟻象'],
 [

In [11]:
import re

rep = {"condition1": "", "condition2": "text"} # define desired replacements here


rep = dict((re.escape(k), v) for k, v in rep.items()) 
pattern = re.compile("|".join(rep.keys()))
text = pattern.sub(lambda m: rep[re.escape(m.group(0))], "(condition1) and --condition2--")
text

'() and --text--'

In [46]:
# fin the most used name

# dic_pest=[]
# for kind in list_pest:
#     kind_dict = dict.fromkeys(kind, 0)
#     dic_pest.append(kind_dict)

    
for ele in table_train['title']:
    
    ele.count('')

In [36]:
dic_pest

[{'太平洋臀紋粉介殼蟲': 0},
 {'番茄黃化捲葉病毒': 0, '番茄捲葉病': 0},
 {'番茄斑點萎凋病毒': 0},
 {'瓜類退綠黃化病毒': 0},
 {'霍香薊黃脈病毒': 0},
 {'褪綠斑駁病毒病': 0},
 {'臺灣本土黑螞蟻': 0, '琉璃蟻': 0},
 {'細菌性黑斑病': 0, '黑斑病': 0, '鬼頭病': 0, '輪斑病': 0},
 {'水稻縞葉枯病': 0},
 {'水稻水象鼻蟲': 0},
 {'番茄嵌紋病毒': 0},
 {'胡瓜嵌紋病毒': 0},
 {'馬鈴薯病毒Y': 0},
 {'細菌性條斑病': 0},
 {'二點小綠葉蟬': 0},
 {'細菌性斑點病': 0},
 {'細菌性角斑病': 0, '臭葉': 0},
 {'柑橘窄胸天牛': 0},
 {'南瓜捲葉病毒': 0},
 {'白輪盾介殼蟲': 0},
 {'地中海果實蠅': 0},
 {'路易氏始葉螨': 0},
 {'斜紋夜盜蟲': 0, '斜紋夜蛾': 0, '行軍蟲': 0, '黑肚蟲': 0, '夜盜蟲': 0, '黑蟲': 0},
 {'稻縱捲葉蟲': 0, '縱捲葉蟲': 0, '瘤野螟': 0, '捲葉蟲': 0, '葉尾蟲': 0},
 {'東方果實蠅': 0, '蜂仔': 0},
 {'草狀矮化病': 0},
 {'皺縮矮化病': 0},
 {'壯鋏普癭蚋': 0},
 {'咖啡木蠹蛾': 0},
 {'台灣黃毒蛾': 0},
 {'小白紋毒蛾': 0},
 {'草地貪夜蛾': 0, '秋行軍蟲': 0},
 {'胡麻葉枯病': 0},
 {'花姬捲葉蛾': 0},
 {'神澤氏葉螨': 0},
 {'葉鞘腐敗病': 0},
 {'柑橘赤葉螨': 0},
 {'黑條萎縮病': 0},
 {'秀粉介殼蟲': 0},
 {'黑點根腐病': 0},
 {'番茄斑潛蠅': 0},
 {'番茄病毒病': 0},
 {'線蟲白尖病': 0},
 {'亞洲潛葉蠅': 0},
 {'番茄潛旋蛾': 0},
 {'臺灣花薊馬': 0},
 {'立枯絲核菌': 0},
 {'假莖象鼻蟲': 0},
 {'球莖象鼻蟲': 0},
 {'疣胸琉璃蟻': 0},
 {'甜菜夜蛾': 0, '蔥仔管蟲': 0, '管仔蟲': 0},
 {'白背飛蝨':

In [39]:
list_chem

[['純白鏈黴菌素700PCU/g水溶性粉劑', '純白鏈黴菌素水溶性粉劑', '純白鏈黴菌素'],
 ['液化澱粉芽孢桿菌YCMA1可溼性粉劑', '液化澱粉芽孢桿菌'],
 ['施亞磷酸溶液；亞磷酸；氫氧化鉀',
  '亞磷酸；氫氧化鉀；溶氫氧化鉀',
  '亞磷酸氫氧化鉀中和之溶液',
  '亞磷酸氫氧化鉀中和之溶液',
  '亞磷酸混和氫氧化鉀',
  '亞磷酸+氫氧化鉀',
  '亞磷酸混氫氧化鉀'],
 ['克熱淨(烷苯磺酸鹽)可濕性粉劑', '克熱淨(烷苯磺酸鹽)'],
 ['加保利可濕性粉劑可濕性粉劑', '加保利可溼性粉劑水懸劑', '加保利可濕性粉劑稀釋', '加保利可濕性粉劑', '加保利'],
 ['克熱淨可濕性粉劑1500倍', '克熱淨可濕性粉劑', '克熱淨溶液', '克熱淨'],
 ['可濕性硫磺水分散性粒劑稀釋',
  '可濕性硫磺可濕性粉劑稀釋3',
  '可濕性硫磺可濕性粉劑稀釋',
  '可濕性硫磺水分散性粒劑',
  '可濕性硫磺粉系列葯劑',
  '可濕性硫磺粉劑',
  '可濕性硫磺'],
 ['鹼性氯氧化銅水份散性粒劑', '鹼性氯氧化銅可濕性粉劑', '鹼性氯氧化銅'],
 ['達滅芬可濕性粉劑或水懸劑', '達滅芬可濕性粉劑', '達滅芬'],
 ['鋅錳右滅達樂水分散性粒劑', '鋅錳右滅達樂'],
 ['益洛寧可濕性粉劑的為害。', '益洛寧可濕性粉劑', '益洛寧'],
 ['可濕性硫黃水分散性粒劑', '可濕性硫黃可濕性粉劑', '可濕性硫黃粉劑', '可濕性硫黃'],
 ['四氯異苯腈水分散性粒劑', '四氯異苯腈可濕性粉劑', '四氯異苯睛可濕性粉劑', '四氯異苯晴可濕性粉劑', '四氯異苯腈'],
 ['賽普護汰寧水分散性粒劑', '賽普護汰寧'],
 ['賽普待克利水分散性粒劑', '賽普待克利'],
 ['賽座滅水懸劑2000倍', '賽座滅水懸劑', '賽座滅'],
 ['鋅錳座賽胺水分散性粒劑', '鋅錳座賽胺'],
 ['嘉賜貝芬混合可濕性粉劑', '嘉賜貝芬'],
 ['嘉磷塞(異丙胺鹽)溶液', '嘉磷塞異丙胺鹽'],
 ['甲基多保淨可濕性粉劑', '甲基多保淨水懸劑', '甲基多保淨軟膏劑', '甲基多保淨'],
 ['嘉賜三賽唑可濕性粉劑', '嘉賜三塞唑可濕性粉劑', '加賜三

In [40]:
list_crop

[['玉荷包荔枝', '荔枝'],
 ['橙蜜香番茄', '橙果小番茄', '小果番茄', '番茄'],
 ['晚崙西亞橙', '晚崙西亞', '丁香'],
 ['禾本科雜草'],
 ['小花蔓澤蘭', '薇甘菊'],
 ['心型薇甘菊', '蔓澤蘭'],
 ['麻豆文旦', '文旦柚', '文旦'],
 ['十字花科'],
 ['台灣欒樹', '欒樹'],
 ['草莓苗期', '草莓'],
 ['鳳梨釋迦', '釋迦'],
 ['明尼吉柚'],
 ['景觀樹木'],
 ['綠肥作物'],
 ['可可椰子'],
 ['結球白菜'],
 ['蔓性作物'],
 ['園藝作物'],
 ['青江白菜'],
 ['包心白菜'],
 ['包心芥菜'],
 ['觀葉植物'],
 ['結球菜類'],
 ['飲料作物'],
 ['青梗白菜'],
 ['天南星科'],
 ['盤固拉草'],
 ['觀賞植物'],
 ['無患子科'],
 ['龍鬚菜'],
 ['洋香瓜'],
 ['落花生', '花生'],
 ['中國梨', '高接梨', '梨'],
 ['番石榴', '芭樂'],
 ['印度棗'],
 ['馬鈴薯'],
 ['紅龍果'],
 ['瓜果類'],
 ['豆菜類'],
 ['洋桔梗'],
 ['百香果'],
 ['番荔枝'],
 ['茂谷柑'],
 ['水蜜桃'],
 ['金針菜', '金針'],
 ['西施柚'],
 ['蔬果類'],
 ['葡萄柚'],
 ['葉菜類'],
 ['葫蘆科'],
 ['芸香科'],
 ['花椰菜'],
 ['聖誕紅'],
 ['薰衣草'],
 ['多花菊'],
 ['唐菖蒲', '唐菖浦'],
 ['花胡瓜'],
 ['九層塔'],
 ['非洲菊'],
 ['佛手瓜'],
 ['七里香', '月桔'],
 ['烏柑仔'],
 ['芥藍菜'],
 ['火鶴花'],
 ['蝴蝶蘭'],
 ['茭白筍', '茭白荀', '筊白筍'],
 ['甜蜜桃'],
 ['小白菜'],
 ['青花菜'],
 ['花菜類'],
 ['草花類'],
 ['盆花類'],
 ['空心菜'],
 ['向日葵'],
 ['旋花科'],
 ['睡蓮科'],
 ['茄果類'],
 ['瓜菜類'],
 ['曼陀羅'],
 ['狼尾草'],
 ['果菜類'],
 ['洛神葵'],


In [45]:
"123".count('1')

1